## Tutorial on How to Use the Dataset for Training
This tutorial provides step-by-step instructions for using the dataset for training a model. The dataset is available now in the same directory.
"""
1. **Split the data into training and testing sets**: Split the dataset into training and testing sets using appropriate techniques such as k-fold cross-validation.
2. **Train the model**: Train a machine learning model on the training set using an appropriate algorithm or framework.
3. **Evaluate the model**: Evaluate the performance of the trained model on the testing set using appropriate metrics such as accuracy, precision, recall, and F1-score.
4. **Make predictions**: Use the trained model to make predictions on new data.
By following these steps, you can use the dataset for training a model and gain valuable insights into your data.

### Load the datdataset
Here you just need to load the dataset using the Huggingfac's datasets library. The dataset is available in the same directory as this notebook.

In [2]:
from datasets import load_from_disk

full_dataset = load_from_disk("my_ner_dataset/sentence_split")
print(full_dataset)

DatasetDict({
    train: Dataset({
        features: ['file_name', 'Tokens', 'ner_tags', 'Labels', 'number_of_tokens', 'Language', 'source', 'Label_counts', 'number_of_annotations', 'sentence_id'],
        num_rows: 2722
    })
    test: Dataset({
        features: ['file_name', 'Tokens', 'ner_tags', 'Labels', 'number_of_tokens', 'Language', 'source', 'Label_counts', 'number_of_annotations', 'sentence_id'],
        num_rows: 319
    })
})


### Visualize the Huggingface dataset
The datase's entries can be shown using the following code:

In [6]:
import pandas as pd

In [10]:
df = pd.DataFrame.from_dict(full_dataset['test'][0:5])
df.head()

,file_name,Tokens,ner_tags,Labels,number_of_tokens,Language,source,Label_counts,number_of_annotations,sentence_id
0,97dda154-93d3-4685-beff-9124e7346d68.txt,"[Title, :, \n , Abundance, of, different, micr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",57,en,BonaRes,Counter(),0,97dda154-93d3-4685-beff-9124e7346d68-0
1,97dda154-93d3-4685-beff-9124e7346d68.txt,"[Abstract_text_2, :, \n , Fertilizer, experime...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-cropSpe...",23,en,BonaRes,"Counter({'startTime': 2, 'cropSpecies': 1, 'en...",4,97dda154-93d3-4685-beff-9124e7346d68-1
2,97dda154-93d3-4685-beff-9124e7346d68.txt,"[The, fields, investigated, were, Wanna, Sand,...","[0, 0, 0, 0, 5, 6, 0, 5, 0, 0, 5, 6, 0, 0, 0, ...","[O, O, O, O, B-soilTexture, I-soilTexture, O, ...",31,en,BonaRes,Counter({'soilTexture': 8}),8,97dda154-93d3-4685-beff-9124e7346d68-2
3,97dda154-93d3-4685-beff-9124e7346d68.txt,"[On, the, WS, and, WM, fields, ,, 30, m³, of, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18,en,BonaRes,Counter(),0,97dda154-93d3-4685-beff-9124e7346d68-3
4,97dda154-93d3-4685-beff-9124e7346d68.txt,"[Starter, fertilization, was, carried, out, wi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",23,en,BonaRes,Counter(),0,97dda154-93d3-4685-beff-9124e7346d68-4


In [23]:
label_list = ["O","B-soilReferenceGroup","I-soilReferenceGroup", "B-soilOrganicCarbon", "I-soilOrganicCarbon", "B-soilTexture", "I-soilTexture",
               "B-startTime", "I-startTime", "B-endTime", "I-endTime", "B-city", "I-city", "B-duration", "I-duration", "B-cropSpecies", "I-cropSpecies",
                 "B-soilAvailableNitrogen", "I-soilAvailableNitrogen", "B-soilDepth", "I-soilDepth", "B-region", "I-region", "B-country", "I-country",
                   "B-longitude", "I-longitude", "B-latitude", "I-latitude", "B-cropVariety", "I-cropVariety", "B-soilPH", "I-soilPH",
                     "B-soilBulkDensity", "I-soilBulkDensity"]

As seen in the above table the dataset contains sentences already split into to words and their respective tags. We will use this to train our model.

### Align and tokenize dataset
The dataset is split into training and test dataset. The goal is to fine-tune an encoder model using the datatset and evaluate the results using the test split. But for each modee, there exists a tokenizer that split the words according to the models preferred inputs. The follwoing function shsould do this.

In [14]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

Here is the application of this function

In [17]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
inputs = tokenizer(full_dataset['test'][2]["Tokens"], is_split_into_words=True)
print(inputs.tokens())
labels = full_dataset['test'][2]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

['[CLS]', 'The', 'fields', 'investigated', 'were', 'Wanna', 'Sand', '(', 'W', '##S', ')', ',', 'Wanna', 'Sand', 'without', 'bio', '##gas', 's', '##lu', '##rry', '(', 'W', '##S', '##o', ')', ',', 'R', '##ost', '##ock', 'Sand', '(', 'RS', ')', 'and', 'Wanna', 'Mars', '##ch', '(', 'W', '##M', ')', '.', '[SEP]']
[0, 0, 0, 0, 5, 6, 0, 5, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0, 5, 6, 0, 5, 0, 0, 5, 6, 0, 5, 0, 0]
[-100, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 5, 6, 6, 6, 0, 5, 0, 0, 5, 6, 6, 0, 5, 6, 0, 0, -100]


The following function transforms the whole dataset to a dataset that is ready to be used as an input to the model

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["Tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [21]:
tokenized_datasets = full_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=full_dataset["train"].column_names,
)

Map:   0%|          | 0/2722 [00:00<?, ? examples/s]

Map: 100%|██████████| 319/319 [00:00<00:00, 12241.94 examples/s]


In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2722
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 319
    })
})

### Evaluation criteria

The **F₁ score** is a statistical measure that balances **precision** and **recall** — two key metrics used to evaluate the performance of models like those used in **Named Entity Recognition (NER)**.

It’s particularly useful in NER because we care about **both** correctly identifying entities (**precision**) and finding **all** relevant entities (**recall**).

---

#### Formula

$$
F_1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}
$$

where

$$
Precision = \frac{TP}{TP + FP}, \qquad
Recall = \frac{TP}{TP + FN}.
$$

---

#### 📘 Definitions in the Context of NER

| Term | Meaning in NER |
|------|----------------|
| **TP (True Positive)** | The model correctly identifies an entity span with the correct label (e.g., correctly detecting “Berlin” as a `LOCATION`). |
| **FP (False Positive)** | The model predicts an entity that does not exist in the gold annotations or assigns the wrong label (e.g., predicting “Berlin” as `PERSON`). |
| **FN (False Negative)** | The model fails to detect a true entity present in the gold annotations. |

---


The following function computes the f-1 score for the model during training and evaluation

In [24]:
import numpy as np
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

### Defining the model
Now comes the time to load the model for training

In [25]:
# Define model attributes
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

from transformers import AutoModelForTokenClassification
# Load the pretrained BERT model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
from huggingface_hub import notebook_login

notebook_login()

### Defining the trainer 
Huggingface provides an easy to use trainer and can be loaded with the training arguments as follows

In [ ]:
from transformers import TrainingArguments
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
args = TrainingArguments(
    "bert-finetuned-ner", # Where to save the model
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

### Train the model

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)
trainer.train()

### Use the model 
After the model is trained and saved, you can load the best model check-point and use it to predict on new text

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "" # Fill with the saved model
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)